```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Static Layer](#static-layer)**


# **FLOW DIAGRAMS**

---

[![](https://mermaid.ink/img/pako:eNpdj8FOwzAMQH8l8mmTNj6gByQQHCbtRI_NNLmJ20a0TuU4oDLt3wkMpIJPlv3sZ1_ARU9QQTfGdzegqDHHF8umRMptLzgP5sBzVnPEheTWqWXTPHxkoccxtqet2e-NhcADtkGRNZ1nkvOUObgw4xh0uXPpzULh7k3dbJJiH7jfnm7biP0_Ya2owf0xfs8ems3KYorFWl57TLuYhB2VTKgPkVcO2MFEMmHw5d3LV9mCDjSRhaqkHuXVguVr4fLsUenZB40CVYdjoh1g1lgv7KBSyfQLPQUsN08_1PUTQWBx9w)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNpdj8FOwzAMQH8l8mmTNj6gByQQHCbtRI_NNLmJ20a0TuU4oDLt3wkMpIJPlv3sZ1_ARU9QQTfGdzegqDHHF8umRMptLzgP5sBzVnPEheTWqWXTPHxkoccxtqet2e-NhcADtkGRNZ1nkvOUObgw4xh0uXPpzULh7k3dbJJiH7jfnm7biP0_Ya2owf0xfs8ems3KYorFWl57TLuYhB2VTKgPkVcO2MFEMmHw5d3LV9mCDjSRhaqkHuXVguVr4fLsUenZB40CVYdjoh1g1lgv7KBSyfQLPQUsN08_1PUTQWBx9w)

Required steps:

1. **[AzureBlob]()**: contains the static `"inhabitants per municipality"` data collected by VWS to be used for further processing.
2. **[Staging](#input-layer)**: inserts the raw `Source` data within the `[VWSSTAGE].[VWS_INHABITANTS_PER_GM_BY_VR]` table.
3. **[Inhabitants Per Municipality By Safety Region](#static-layer)**: inserts the `Staging` data and cast the correct `Database Types` within `[VWSSTATIC].[VWS_INHABITANTS_PER_GM_BY_VR]` using business logic implemented in `[dbo].[SP_INSERT_SL_VWS_INHABITANTS_PER_GM_BY_VR]`.

- **<font color=teal>SL</font>**: Static Layer


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Functions.ipynb",
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/DataFactory/Utils/MunicipalityRemapping.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[VWS_INHABITANTS_PER_GM_BY_VR]'))
CREATE TABLE [VWSSTAGE].[VWS_INHABITANTS_PER_GM_BY_VR] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[REGION] [VARCHAR](50) NOT NULL,
	[GM_CODE] [VARCHAR](50) NOT NULL,
	[VR_CODE] [VARCHAR](50) NOT NULL,
	[VR_NAME] [VARCHAR](50) NOT NULL,
	[INHABITANTS] [VARCHAR](50) NOT NULL
);
GO

# **STATIC LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[VWS_INHABITANTS_PER_GM_BY_VR]'))
CREATE TABLE [VWSSTATIC].[VWS_INHABITANTS_PER_GM_BY_VR] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [REGION] [VARCHAR](50) NOT NULL,
	[GMCODE] [VARCHAR](50) NOT NULL,
	[VRCODE] [VARCHAR](50) NOT NULL,
	[VRNAME] [VARCHAR](50) NOT NULL,
	[INHABITANTS] [INT] NOT NULL
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_VWS_INHABITANTS_PER_GM_BY_VR]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[VWS_INHABITANTS_PER_GM_BY_VR] (
        [REGION], 
        [GMCODE], 
        [VRCODE], 
        [VRNAME], 
        [INHABITANTS]
    )
    SELECT
        ISNULL(T2.[GM_NAME_NEW],T1.[REGION]),
        ISNULL(T2.[GM_CODE_NEW],T1.[GM_CODE]),
        ISNULL(T2.[VR_CODE],T1.[VR_CODE]),
        ISNULL(T2.[VR_NAME],T1.[VR_NAME]),
        CAST(ROUND(SUM(CAST(T1.[INHABITANTS] AS INT) * ISNULL(T2.[SHARES], 1.0)),0) AS INT)
    FROM
        [VWSSTAGE].[VWS_INHABITANTS_PER_GM_BY_VR] T1
    LEFT JOIN (
        SELECT 
            * 
        FROM [VWSSTATIC].[VWS_MUNICIPALITY_REMAPPING] WITH(NOLOCK)
        WHERE [ACTIVE] = 1
            AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[VWS_MUNICIPALITY_REMAPPING] WITH(NOLOCK))  
    ) T2 ON T1.[GM_CODE] = T2.[GM_CODE_ORIGINAL]
    WHERE T1.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[VWS_INHABITANTS_PER_GM_BY_VR])
    GROUP BY 
        ISNULL(T2.[GM_NAME_NEW],T1.[REGION]),
        ISNULL(T2.[GM_CODE_NEW],T1.[GM_CODE]),
        ISNULL(T2.[VR_CODE],T1.[VR_CODE]),
        ISNULL(T2.[VR_NAME],T1.[VR_NAME])
END;
GO